**YOUR NAMES HERE**

Fall 2019

CS 343: Neural Networks

Project 2: Multi-layer Perceptrons

**Draft due 11:59pm Thurs Sept 26**


Summary of files in this project:
- `softmax_layer.ipynb`
- `single_layer_net.py`
- `preprocess_data.py`
- `mlp.ipynb`
- `mlp.py`

**REMINDER**: Submit rubric on Google Classroom

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# for obtaining the STL-dataset
import load_stl10_dataset

# for preprocessing dataset
import preprocess_data

# Set the color style so that Professor Layton can see your plots
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
# Make the font size larger
plt.rcParams.update({'font.size': 20})

# Turn off scientific notation when printing
np.set_printoptions(suppress=True, precision=3)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

## Data

### a. STL-10

**TODO**: Run the cell below, to preprocess STL-10 dataset like you did in the other notebook.

In [ ]:
# Download the dataset from the internet, convert it to Numpy ndarray, resize to 32x32
# cache it locally on your computer for faster loading next time.
stl_imgs, stl_labels = load_stl10_dataset.load()

# Load in the string names for each class
classes = np.loadtxt(os.path.join('data', 'stl10_binary', 'class_names.txt'), dtype=str)

# Preprocess image pixel values for the MLP net
stl_imgs, stl_labels = preprocess_data.preprocess_stl(stl_imgs, stl_labels)
print(f'stl_imgs dtype is {stl_imgs.dtype} and it should be float64')
print(f'stl_imgs max is {np.max(stl_imgs[:, 1:]):.3f} and it should be 0.668')
print(f'stl_imgs shape is {stl_imgs.shape} and it should be (5000, 3072)')
print(f'stl_labels span {stl_labels.min()}->{stl_labels.max()} and it should be 0->9')
    
x_train, y_train, x_test, y_test, x_val, y_val, x_dev, y_dev = preprocess_data.create_splits(stl_imgs, stl_labels)  
print ('Train data shape: ', x_train.shape)
print ('Train labels shape: ', y_train.shape)
print ('Test data shape: ', x_test.shape)
print ('Test labels shape: ', y_test.shape)
print ('Validation data shape: ', x_val.shape)
print ('Validation labels shape: ', y_val.shape)
print ('dev data shape: ', x_dev.shape)
print ('dev labels shape: ', y_dev.shape)

### Circle in a square

**TODO** Run the code below from the other notebook to load in the CIS dataset

In [ ]:
val_size = 20

cis_train_path = os.path.join('data', 'cis', 'cis_train.dat')
cis_test_path = os.path.join('data', 'cis', 'cis_test.dat')

cis_train_all = np.loadtxt(cis_train_path, delimiter='\t')

# shuffle the data
s_inds = np.arange(len(cis_train_all))
cis_train_all = cis_train_all[s_inds]

cis_train_x = cis_train_all[:, :2]
cis_train_y = cis_train_all[:, 2].astype(int)

cis_val_x = cis_train_x[:val_size]
cis_train_x = cis_train_x[val_size:]
cis_val_y = cis_train_y[:val_size]
cis_train_y = cis_train_y[val_size:]

cis_test_all = np.loadtxt(cis_test_path, delimiter='\t')
cis_test_x = cis_test_all[:, :2]
cis_test_y = cis_test_all[:, 2].astype(int)

print ('CIS Train data shape: ', cis_train_x.shape)
print ('CIS Train labels shape: ', cis_train_y.shape)
print ('CIS Validation data shape: ', cis_val_x.shape)
print ('CIS Validation labels shape: ', cis_val_y.shape)
print ('CIS Test data shape: ', cis_test_x.shape)
print ('CIS Test labels shape: ', cis_test_y.shape)

### b. Circle in a square dataset

**Properties:**
- 2 classes of features

**TODO**:
1. Download the circle in a square dataset. Folder structure is `<project folder>/data/cis/<cis dat files>`
2. Create numpy arrays for the train/test data and separate variables for the associated labels by running the below code

In [ ]:
val_size = 20

cis_train_path = os.path.join('data', 'cis', 'cis_train.dat')
cis_test_path = os.path.join('data', 'cis', 'cis_test.dat')

cis_train_all = np.loadtxt(
cis_train_path, delimiter='\t')

cis_train_x = cis_train_all[:, :2]
cis_train_y = cis_train_all[:, 2].astype(int)

cis_val_x = cis_train_x[:val_size]
cis_train_x = cis_train_x[val_size:]
cis_val_y = cis_train_y[:val_size]
cis_train_y = cis_train_y[val_size:]

cis_test_all = np.loadtxt(cis_test_path, delimiter='\t')
cis_test_x = cis_test_all[:, :2]
cis_test_y = cis_test_all[:, 2].astype(int)

print ('CIS Train data shape: ', cis_train_x.shape)
print ('CIS Train labels shape: ', cis_train_y.shape)
print ('CIS Validation data shape: ', cis_val_x.shape)
print ('CIS Validation labels shape: ', cis_val_y.shape)
print ('CIS Test data shape: ', cis_test_x.shape)
print ('CIS Test labels shape: ', cis_test_y.shape)

Below, you should see a...black circle in a white unit square :) 

In [ ]:
plt.scatter(cis_test_x[:,0], cis_test_x[:,1], c=cis_test_y)
plt.axis('equal')
plt.show()

## Task 3) Implement Multilayer Perceptron (MLP) with softmax activation and cross-entropy loss

Now that we've tested the softmax activation function and cross-entropy loss functions in a single-layer net, let's implement the MLP version.

Much of your work on the single layer net will carry over, so go ahead and copy-paste and modify as needed!

The structure of our MLP will be:

Input layer (X units) -> Hidden layer (Y units) with Rectified Linear activation (ReLu) -> Output layer (Z units) with softmax activation

### a. Implement the following functions in `mlp.py`

- `initialize_wts`
- `accuracy`
- `one_hot`
- `predict`
- `forward`
- `backward`
- `fit`

### b. Test key functions with randomly generated data

In [ ]:
from mlp import MLP

In [ ]:
# Create a dummy net for debugging
num_inputs = 3
num_features = 6
num_hidden_units = 7
num_classes = 5

net = MLP(num_features, num_hidden_units, num_classes)

In [ ]:
# Generate random data and classes
np.random.seed(0)
test_x = np.random.normal(loc=0, scale=100, size=(num_inputs, num_features))
test_y = np.random.uniform(low=0, high=num_classes-1, size=(num_inputs,))
test_y = test_y.astype(int)
print(f'Test input shape: {test_x.shape}')
print(f'Test class vector shape: {test_y.shape}')

Test `initialize_wts`

In [ ]:
net.initialize_wts(M=num_features, H=num_hidden_units, C=num_classes, std=0.01)
print(f'y wt shape is {net.y_wts.shape} and should be (6, 7)')
print(f'y bias shape is {net.y_b.shape} and should be (7,)')
print(f'z wt shape is {net.z_wts.shape} and should be (7, 5)')
print(f'z bias shape is {net.z_b.shape} and should be (5,)')

print(f'1st few y wts are\n{net.y_wts[:,0]}\nand should be\n[ 0.018 -0.002  0.004  0.007  0.015  0.002]')
print(f'y bias is\n{net.y_b}\nand should be\n[-0.017  0.02  -0.005 -0.004 -0.013  0.008 -0.016]')
print(f'1st few z wts are\n{net.z_wts[:,0]}\nand should be\n[-0.002 -0.    -0.004  0.002  0.001  0.004  0.001]')
print(f'z bias is\n{net.z_b}\nand should be\n[ 0.015  0.019  0.012 -0.002 -0.011]')

Test the `predict` method

In [ ]:
test_y_pred = net.predict(test_x)
print(f'Predicted classes are {test_y_pred} and should be [3 0 0]')

Test the `forward` method focusing on`ReLU`(net act of hidden layer `y`)

In [ ]:
_,y_net_act_test,_,_,_ = net.forward(test_x, test_y)

correct_y_act = np.array([[7.66 , 4.47 , 0.804, 9.981, 0.   , 0.   , 0.   ],
       [2.37 , 2.717, 2.18 , 2.552, 0.357, 0.   , 0.   ],
       [3.997, 2.671, 1.195, 3.034, 0.   , 0.   , 0.   ]])

print(f'Your y activation is\n{y_net_act_test}')
print(f'The correct y activation (ReLU) is\n{y_net_act_test}')

Test the `forward` method

In [ ]:
_,_,_,probs,_ = net.forward(test_x, test_y)

correct_probs = np.array([[0.219, 0.2  , 0.191, 0.219, 0.171],
       [0.208, 0.204, 0.201, 0.205, 0.183],
       [0.208, 0.202, 0.202, 0.205, 0.183]])

print(f'Your z activation (class probabilities) is\n{probs}')
print(f'The correct z activation (class probabilities) is\n{correct_probs}')
print(f'The sums across rows (for each data sample) are {np.sum(probs, axis=1)}.')
print(f'  You should know what should be :)')

Test the `forward` method, focusing on loss

In [ ]:
y_in, y_act ,z_in, z_act, loss = net.forward(test_x, test_y)
correct_loss = 1.564402690536365

print(f'Your average loss is\n{loss}')
print(f'The correct average loss is approx\n{correct_loss}')

Test the `forward` method, focusing on regularization

In [ ]:
y_in, y_act ,z_in, z_act, loss = net.forward(test_x, test_y, reg=1000)
correct_loss = 5.257207314928798

print(f'Your regularized average loss is\n{loss}')
print(f'The correct regularized average loss is approx\n{correct_loss}')

Test the `backward` method

In [ ]:
y_in, y_act ,z_in, z_act, loss = net.forward(test_x, test_y, reg=0.5)
grads = net.backward(test_x, test_y, y_in, y_act ,z_in, z_act, reg=0.5)

print('Your gradient for y_wts is\n', grads[0])
print('Your gradient for y_b is\n', grads[1])
print('Your gradient for z_wts is\n', grads[2])
print('Your gradient for z_b is\n', grads[3])

The correct gradients are:

`
Your gradient for y_wts is
 [[-0.476  0.057 -0.458 -0.115  0.03  -0.005  0.005]
 [-0.002  0.014 -0.046 -0.162  0.004  0.004  0.001]
 [-0.088  0.038 -0.166 -0.325 -0.001 -0.004 -0.013]
 [-0.331  0.067 -0.398 -0.332  0.001  0.    -0.001]
 [-0.318  0.089 -0.465 -0.615 -0.001 -0.01  -0.002]
 [-0.315 -0.036 -0.036  0.806  0.029 -0.005 -0.007]]
Your gradient for y_b is
 [-0.005  0.    -0.004 -0.     0.     0.     0.   ]
Your gradient for z_wts is
 [[-2.879  0.933  0.131  0.987  0.816]
 [-1.69   0.669 -0.261  0.699  0.584]
 [-0.374  0.278 -0.45   0.284  0.242]
 [-3.221  1.041  0.154  1.111  0.904]
 [ 0.024  0.027 -0.091  0.029  0.015]
 [ 0.002 -0.003 -0.004 -0.003 -0.002]
 [ 0.    -0.006  0.005  0.002 -0.008]]
Your gradient for z_b is
 [-0.455  0.202 -0.135  0.209  0.179]
`

Test loss over epoch. 

The below code should generate a curve that rapidly drops to 0 (there might be fluctuations and it might not be monotonic and that's ok)

In [ ]:
net = MLP(num_features, num_hidden_units, num_classes)
loss, acc_t, acc_v = net.fit(test_x, test_y, test_x, test_y, reg=0, lr=0.001, mini_batch_sz=3, n_epochs=30)

plt.plot(loss)
plt.xlabel('Iteration')
plt.ylabel('Avg cross-entropy Loss')
plt.show()

### c. Test MLP with Circle in Square dataset

First test case where training data = test data.

You should see a nice drop and plateau in loss (after a bunch of print outs).

In [ ]:
# train = test
hidden_size = 50
net = MLP(cis_train_x.shape[1], hidden_size, 2)
loss, acc_t, acc_v = net.fit(cis_test_x, cis_test_y, cis_val_x, cis_val_y, reg=0, lr=0.01, mini_batch_sz=100, n_epochs=700)

plt.plot(loss)
plt.xlabel('Iteration')
plt.ylabel('Avg cross-entropy Loss')
plt.show()

**TODO**: Run the below cell after training and generate the CIS plot. You should see a well-defined circle inside a square region.

In [ ]:
cis_test_y_pred = net.predict(cis_test_x)
plt.scatter(cis_test_x[:,0], cis_test_x[:,1], c=cis_test_y_pred)
plt.axis('equal')
plt.show()

**Second test case where training data != test data:** You should see a jagged polygon approximation to a circle inside a square region. 

In [ ]:
# train != test
net = MLP(cis_train_x.shape[1], hidden_size, 2)
loss, acc_t, acc_v = net.fit(cis_train_x, cis_train_y, cis_val_x, cis_val_y, reg=0, lr=0.1, mini_batch_sz=len(cis_train_x), n_epochs=10000)

plt.plot(loss)
plt.xlabel('Iteration')
plt.ylabel('Avg cross-entropy Loss')
plt.show()

**TODO**: Run the below cell after training and generate the CIS plot.

In [ ]:
cis_test_y_pred = net.predict(cis_test_x)
plt.scatter(cis_test_x[:,0], cis_test_x[:,1], c=cis_test_y_pred)
plt.axis('equal')
plt.show()

**Question 5**: How do you interpret the circle-in-square results? Do you think the single-layer net (with softmax) can handle the CIS dataset? Why or why not? (You're invited to try it, maybe as an extension :)

**Question 6**: Play with # hidden units, epochs, regularization strength on CIS training...how does each parameter affect the results?

### d. Train on STL-10 dataset, plot performance

Try training the MLP on the below set of hyperparameters

In [ ]:
net = MLP(x_train.shape[1], 100, 10)
loss, acc_t, acc_v = net.fit(x_train, y_train, x_val, y_val,
                             lr=0.05, reg=0.01, mini_batch_sz=500, n_epochs=50, verbose=0)
plt.plot(loss)
plt.xlabel('Training iteration')
plt.ylabel('Avg loss (cross entropy)')
plt.title('STL-10 loss')
plt.show()

In [ ]:
plt.plot(acc_t, label='Training accuracy')
plt.plot(acc_v, label='Validation accuracy')
plt.xlabel('Training iteration')
plt.ylabel('Avg loss (cross entropy)')
plt.title('STL-10 accuracy')
plt.legend()
plt.show()

**Question 7**: What do the above loss and training and validation accuracy curves suggest about the quality of the hyperparameters used during training?

### e. Optimize on STL-10 dataset, plot performance

**TODO** Write code in the cell below to perform a grid search to find the combinations of

- learning rate
- regularization strength
- number hidden units
- mini-batch size

that yields the highest STL-10 validation set accuracy.

You should be able to achieve higher accuracy than in the single layer version of the softmax network.

In [ ]:
best_val_acc = 0
best_lr = None
best_reg = None
best_hidden = None
best_mini_batch_sz = None

**TODO**: Set the parameters below to the best ones found by your grid search.
- Generate and include the Test STL-10 loss curve
- Generate and include the training and validation accuracy curves in the second cell down.

Adjust the number of training epochs as needed.

In [ ]:
# Train the net with the bestparam settings
best_lr = None
best_reg = None
best_hidden = None

bestNet = MLP(x_train.shape[1], best_hidden, 10)
loss, acc_t, acc_v = bestNet.fit(x_test, y_test, x_val, y_val,
                                 lr=best_lr, reg=best_reg, mini_batch_sz=best_mini_batch_sz, n_epochs=5000, verbose=1)
plt.plot(loss)
plt.xlabel('Training iteration')
plt.ylabel('Avg loss (cross entropy)')
plt.title('Test STL-10 data loss')
plt.show()

In [ ]:
plt.plot(acc_t, label='Training accuracy')
plt.plot(acc_v, label='Validation accuracy')
plt.xlabel('Training iteration')
plt.ylabel('Avg loss (cross entropy)')
plt.title('Test STL-10 data accuracy')
plt.legend()
plt.show()

### e. Visualize learned weights

**TODO**: 

- In the cell below, get the weights of your best net's hidden layer (Y), reshape/transpose them so that they are `(N, 32, 32, 3)`.
- Run the `plot_weights` function to generate a grid visualization of them.

In [ ]:
best_y_wts = bestNet.get_y_wts()


In [ ]:
def plot_weights(wts):
    grid_sz = int(np.sqrt(len(wts)))
    plt.figure(figsize=(20,20))
    for x in range(grid_sz):
        for y in range(grid_sz):
            lin_ind = np.ravel_multi_index((x, y), dims=(grid_sz, grid_sz))
            plt.subplot(grid_sz, grid_sz, lin_ind+1)
            currImg = wts[lin_ind]
            low, high = np.min(currImg), np.max(currImg)
            currImg = 255*(currImg - low) / (high - low)
            currImg = currImg.astype('uint8')
            plt.imshow(currImg)
            plt.gca().axis('off')
    plt.show()

In [ ]:
plot_weights(best_y_wts)

## Extensions

**Reminder**: Please do not integrate extensions into your base project so that it changes the expected behavior of core functions. It is better to duplicate the base project and add features from there.

1) Instead of computing the loss over a mini-batch, compute it over epochs instead. Compare and contrast the approaches. Be sure to include analysis figures.

2) Investigate how the single layer softmax network does with the CIS dataset. Explain and provide plots showing your results.

3) If you have time to spare (or want to throw more computing power at the STL-10 dataset), process through the SLP and MLP and tune hyperparameters with the dataset at its original resolution (96x96 images). Show images of your learned weights. Can you find a training sweet spot where the learned weight visualizations look particularly cool?

4) Implement the sigmoid classifer (same network structure, except use sigmoid for netact) with the cross-entropy loss by creating another subclass of `SingleLayerNet` and/or `MLP`. Compare and contrast results achieved by the softmax/cross-entropy network.

5) Explore the effects of batch gradient descent, stochastic gradient descent, and mini-batch gradient descent.

6) Obtain, preprocess, train, and evaluate the performance of `SingleLayerNet` and/or `MLP` on another dataset with comparable types of image features. 